In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import numpy as np
import cv2
import argparse
import face_alignment

default_head_name = 'stephen'

parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
opt_parser = parser.parse_known_args()[0]

img =cv2.imread('images/' + opt_parser.jpg)
img2 = img

predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cpu', flip_input=False)
shapes = predictor.get_landmarks(img)
data = np.array(shapes)
x = data[0, 36:48, 0]
y = data[0, 36:48, 1]

# print(x)
# print(y)

x_L_1 = int(min(x[ :5]))
x_R_1 = int(max(x[ :5]))
y_D_1 = int(min(y[ :5]))
y_U_1 = int(max(y[ :5]))

x_L_2 = int(min(x[6: ]))
x_R_2 = int(max(x[6: ]))
y_D_2 = int(min(y[6: ]))
y_U_2 = int(max(y[6: ]))

# print(x_L_1 , x_R_1 , y_D_1 , y_U_1)
# print(x_L_2 , x_R_2 , y_D_2 , y_U_2)

cv2.rectangle(img , (x_L_1 , y_D_1) , (x_R_1 , y_U_1) , (0 , 255 , 0) , 2)
cv2.rectangle(img , (x_L_2 , y_D_2) , (x_R_2 , y_U_2) , (0 , 255 , 0) , 2)

img_eye_1 = cv2.resize(img2[y_D_1 : y_U_1 , x_L_1 : x_R_1] , (300 , 300))
img_eye_2 = cv2.resize(img2[y_D_2 : y_U_2 , x_L_2 : x_R_2] , (300 , 300))

img_eye_1 = img_eye_1[60:240 , 10:290]
img_eye_2 = img_eye_2[60:240 , 10:290]

img_eye_l_gray = cv2.cvtColor(img_eye_1 , cv2.COLOR_RGB2GRAY)
img_eye_r_gray = cv2.cvtColor(img_eye_2 , cv2.COLOR_RGB2GRAY)

ret1, adaptive_l = cv2.threshold(img_eye_l_gray , 22 , 255 , cv2.THRESH_BINARY_INV)
ret2, adaptive_r = cv2.threshold(img_eye_r_gray , 22 , 255 , cv2.THRESH_BINARY_INV)

mask_l = cv2.erode(adaptive_l , None , iterations=2)
mask_r = cv2.erode(adaptive_r , None , iterations=2)

mask_l = cv2.dilate(mask_l , None , iterations=2)
mask_r = cv2.dilate(mask_r , None , iterations=2)

cnts_l = cv2.findContours(mask_l.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)[-2]
cnts_r = cv2.findContours(mask_r.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)[-2]

from matplotlib import pyplot as plt 

if len(cnts_l) > 0:
    c = max(cnts_l, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(c)
    M = cv2.moments(c)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

    cv2.circle(img_eye_1, (int(x), int(y)), int(radius), (255, 0, 0), 2)
    cv2.circle(img_eye_1, center, 5, (0, 0, 255), -1)

if len(cnts_r) > 0:
    c = max(cnts_r, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(c)
    M = cv2.moments(c)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

    cv2.circle(img_eye_2, (int(x), int(y)), int(radius), (255, 0, 0), 2)
    cv2.circle(img_eye_2, center, 5, (0, 0, 255), -1)
    # cv2.imshow("eye_l", img_eye_1)
    # cv2.imshow("eye_r", img_eye_2)
    # cv2.imshow('video', img)
    # cv2.waitKey(0)
    eye_l_plt = img_eye_1[:, :, : : -1]
    plt.imshow(eye_l_plt)
    plt.show()
    eye_r_plt = img_eye_2[:, :, : : -1]
    plt.imshow(eye_r_plt)
    plt.show()
    img_plt = img[:, :, : : -1]
    plt.imshow(img_plt)
    plt.show()
